# Creating a Story Map Using Leaflet and CSVs

---

**Objectives:**

By the end of this exericse, you should be able to:

* Create an HTML page
* Host an interactive map (Leaflet) on GitHub Pages
* Create a storymap using Python, Leaflet, and CSV files
* Explore MapStore [here](https://mapstore.readthedocs.io/en/latest/user-guide/exploring-stories/).

---

In this exercise, you will learn how to present spatial data in an interactive narrative similar to [ESRI StoryMaps](https://storymaps-classic.arcgis.com/en/gallery/#s=0) using open source products. You will learn how to host a live version of an HTML webpage on GitHub Pages.

**HTML** (HyperText Markup Language) is a markup language used for files meant to be displayed in a web browser. HTML syntax is similar to the `markdown` syntax used in these `jupyter notebooks`. From [Wikipedia](https://en.wikipedia.org/wiki/HTML):

"HTML elements are the building blocks of HTML pages. With HTML constructs, images and other objects such as interactive forms may be embedded into the rendered page. HTML provides a means to create structured documents by denoting structural semantics for text such as headings, paragraphs, lists, links, quotes and other items. HTML elements are delineated by tags, written using angle brackets. Tags such as `<img />` and `<input />` directly introduce content into the page. Other tags such as `<p>` surround and provide information about document text and may include other tags as sub-elements. Browsers do not display the HTML tags but use them to interpret the content of the page."

If you double-click on the `raleigh.html` file in your folder, a map will open in your web browser. Next, let's open the `raleigh.html` file in **VS Code**.

This code is adapted from the `index.html` file from [this GitHub repository](https://github.com/HandsOnDataViz/leaflet-map-simple).

This exercise is an adaptation of the ["Leaflet Storymaps with Google Sheets" tutorial](https://handsondataviz.org/leaflet-storymaps-with-google-sheets.html) from the *Hands-On Data Visualization* book by Jack Dougherty & Ilya Ilyankou. The open-acess web edition of the book can be found [here](https://handsondataviz.org/).

### Host a Live Leaflet Map on GitHub Pages

In order to host the `raleigh.html` document on the internet, you need to create a GitHub repository for this project. The best way to do this is to **fork** the Week8 repo 

# Add more instructions to this later

### A Brief Intro to HTML and Leaflet Markers

Now that we have the HTML file open, let's expore what it means. There are several comments in the document to help our understanding of the document.

"The first block tells web browsers which formatting to apply to the rest of the page of code. The second block instructs the browser to load the Leaflet code library, the open-source software that constructs the interactive map. The third block describes where the map and title should be positioned on the screen. The good news is that you don’t need to touch any of those blocks of code, so leave them as-is. But you do want to modify a few lines further below." [(Ch. 10, Dougherty & Ilyankou, 2022)](https://handsondataviz.org/copy-leaflet.html)

Once we are in the `<body>` section of the HTML document, we are looking at code that can be used to edit our map. You can easily change the code where you see `EDIT` in a comment. 

Take some time to explore changing the `map-title`, the center coordinates, and zoom of the map.

Next, add a point marker with pop-up text for where you live. To do this, copy and paste:

```
  /* Display a point marker with pop-up text */
  L.marker([35.7818, -78.6763]).addTo(map) // EDIT marker coordinates
  .bindPopup("Jordan Hall"); // EDIT pop-up text message
```

below the **Jordan Hall** marker. Edit the coordinates and `.bindPopup()` text to match where you live.

Now, refresh your web browser page with the `raleigh.html` file. **Did your new marker appear?**

### Leaflet Storymap with a CSV

We are now going to start working on our Leaflet Storymap by building a `pandas dataframe` which we can save as a CSV.

Our storymap will have a scrolling narrative that follows points along our map. You will learn how to add images, audio and video files, and different map backgrounds. 

**how to get a "use this template" button on GitHub**



In [ ]:
import geopy
import pandas
from geopy.geocoders import Nominatim

In [ ]:
def main():
    io = pandas.read_csv('census-historic-population-borough.csv', index_col=None, header=0, sep=",")

    def get_latitude(x):
        return x.latitude

    def get_longitude(x):
        return x.longitude

    geolocator = Nominatim(user_agent="email@email.com")

    geolocate_column = io['Area_Name'].apply(geolocator.geocode)
    io['latitude'] = geolocate_column.apply(get_latitude)
    io['longitude'] = geolocate_column.apply(get_longitude)
    io.to_csv('geocoding-output.csv')

if __name__ == '__main__':
  main()

In [1]:
import folium

# Create a Map instance
m = folium.Map(location=[60.25, 24.8], tiles='Stamen Toner',
                   zoom_start=10, control_scale=True)

In [2]:
m

In [3]:
outfp = "./data/base_map.html"
m.save(outfp)

In [11]:
# Let's change the basemap style to 'Stamen Toner'
m = folium.Map(location=[35.7796, -78.6382], tiles='Stamen Toner',
            zoom_start=12, control_scale=True, prefer_canvas=True)

# Filepath to the output
outfp = "./data/base_map2.html"

# Save the map
m.save(outfp)

In [6]:
import geopandas as gpd

# Filepaths
nc_colleges_path = './data/Colleges_and_Universities/Colleges_and_Universities.shp'
nc_amtrak_stations_path = './data/Amtrak_Stations_Trains_Reproj_NC/Amtrak_Stations_Trains_Reproj_NC.shp'

# Read data
nc_colleges = gpd.read_file(nc_colleges_path)
nc_amtrak_stations = gpd.read_file(nc_amtrak_stations_path)

# Print CRS
print(nc_colleges.crs)
print(nc_amtrak_stations.crs)


epsg:32119
epsg:4269


In [8]:
# Reproject to WGS84
nc_colleges_wgs84 = nc_colleges.to_crs(epsg=4326)
nc_amtrak_stations_wgs84 = nc_amtrak_stations.to_crs(epsg=4326)

# Create a Geo-id which is needed by the Folium (it needs to have a unique identifier for each row)
nc_colleges_wgs84['geoid'] = nc_colleges_wgs84.index.astype(str)
nc_amtrak_stations_wgs84['geoid'] = nc_amtrak_stations_wgs84.index.astype(str)

In [18]:
# Create a Clustered map where points are clustered
# marker_cluster = folium.MarkerCluster().add_to(m)

# Create college points on top of the map
for idx, row in nc_colleges_wgs84.iterrows():
    # Get lat and lon of points
    lon = row['geometry'].x
    lat = row['geometry'].y

    # Get college information
    college_name = row['NAME']
    # Add marker to the map
    folium.RegularPolygonMarker(location=[lat, lon], popup=college_name, fill_color='#2b8cbe', number_of_sides=6, radius=8).add_to(m)


In [19]:
# Filepath to the output
outfp = "./data/base_map3.html"

# Save the map
m.save(outfp)

In [20]:
#Define coordinates of where we want to center our map
raleigh_coords = [35.7796, -78.6382]

#Create the map
my_map = folium.Map(location = raleigh_coords, zoom_start = 13)

#Display the map
my_map

In [21]:
# Create college points on top of the map
for idx, row in nc_colleges_wgs84.iterrows():
    # Get lat and lon of points
    lon = row['geometry'].x
    lat = row['geometry'].y

    # Get college information
    college_name = row['NAME']

    #Add markers to the map
    folium.Marker([lat,lon], popup=college_name).add_to(my_map)

my_map

In [22]:
my_map.save('./data/raleigh_colleges.html')